In [ ]:
import cv2
import numpy as np
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from skimage import color
import matplotlib.pyplot as plt



In [ ]:
# Load the image and move to GPU
img = cv2.imread(r'C:\Users\You\Desktop\11.25\WPS1\3.png')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
img_gpu = torch.tensor(img, dtype=torch.float32, device='cuda') / 255.0  # Normalize

In [ ]:
# Get image size
rows, cols, _ = img.shape

# Split the RGB channels
r, g, b = img_gpu[:, :, 0], img_gpu[:, :, 1], img_gpu[:, :, 2]

# Convert to HSI color space
den = torch.sqrt((r - g) ** 2 + (r - b) * (g - b)) + 1e-6  # Avoid division by zero
theta = torch.acos(0.5 * ((r - g) + (r - b)) / den)
H = theta.clone()
H[b > g] = 2 * np.pi - H[b > g]
H = H / (2 * np.pi)

S = 1 - 3 * torch.min(torch.min(r, g), b) / (r + g + b + 1e-6)
I = (r + g + b) / 3

# Convert to LAB color space using skimage
lab_img = color.rgb2lab(img / 255.0)
Lab_l = torch.tensor(lab_img[:, :, 0], dtype=torch.float32, device='cuda')
Lab_a = torch.tensor(lab_img[:, :, 1], dtype=torch.float32, device='cuda')
Lab_b = torch.tensor(lab_img[:, :, 2], dtype=torch.float32, device='cuda')

# Stack features (RGB, HSI, LAB) into a tensor
features = torch.stack([r, g, b, H, S, I, Lab_l, Lab_a, Lab_b], dim=2)


In [ ]:
# Define a neural network (assuming it is already trained and available)
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(9, 16),
            nn.ReLU(),
            nn.Linear(16, 1)
        )
    
    def forward(self, x):
        return self.fc(x)

# Load the pre-trained model
net = SimpleNN().to('cuda')
net.eval()


In [ ]:
# Predict moisture values
moisture_values = torch.zeros((rows, cols), device='cuda')
for i in range(rows):
    for j in range(cols):
        pixel_features = features[i, j, :].view(1, -1)  # Reshape for NN input
        predicted_value = net(pixel_features).item()
        moisture_values[i, j] = predicted_value

# Move results to CPU for visualization
moisture_values = moisture_values.cpu().numpy()

# Normalize moisture values (clamp between 0 and 100)
moisture_values = np.clip(moisture_values, 0, 100)

# Plot the original image
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(img)
plt.title("Original Image")
plt.axis("off")

# Plot the moisture heatmap
plt.subplot(1, 2, 2)
plt.imshow(moisture_values, cmap='jet')
plt.colorbar(label="Moisture Content")
plt.title("Moisture Heatmap")
plt.axis("off")

plt.show()